In [1]:
from pathlib import Path
import pandas as pd
import re

In [2]:
path = Path(r"C:\Users\stapi\PycharmProjects\news_scrape\outstanding_shares_text.txt")

In [3]:
with open(path) as file:
    text = file.read()

In [4]:
raw_text_list = text.split("(")

In [5]:
filter_one = []
for words in raw_text_list:
    if "http" in words:
        filter_one.append(words)

In [6]:
search_text = "Shares Outstanding"
index_list = []

for text in filter_one:
    index = text.find(search_text)
    index_list.append(f"{text[:95]},{text[index:index+50]}")
    
print(index_list[0])

'https://www.sec.gov/ix?doc=/Archives/edgar/data/66740/000006674022000076/mmm-20220930.htm','En,Shares Outstanding
As of 09/30/2022
552,742,915
Re


In [14]:
url_list = []
figures_list = []
for text in index_list:
    url_list.append(text.split(",")[0])
    figures_list.append(text.split("\n")[1:])

print(len(url_list))
print(len(figures_list))

499
499


In [19]:
symbols_columns = []
for row in url_list:
    symbols_columns.append(row.split("-")[0].split("/")[-1].upper())
print(len(symbols_columns))

499


In [18]:
smooth_one = []
for rows in symbols_columns:
    if "2" in rows:
        smooth_one.append(rows.split("2")[0])
    else:
        smooth_one.append(rows)
print(len(smooth_one))

499


In [21]:
figures_list[0]

['As of 09/30/2022', '552,742,915', 'Re']

In [36]:
outstanding_shares = []
for i, row in enumerate(figures_list):
    outstanding_shares.append(row[1])

print(len(outstanding_shares))

499


In [126]:
data = {"Symbols":smooth_one, "Source_urls":url_list, "Oustanding_Shares":outstanding_shares}
df = pd.DataFrame(data, index=smooth_one)

In [127]:
df.head()

,Symbols,Source_urls,Oustanding_Shares
MMM,MMM,'https://www.sec.gov/ix?doc=/Archives/edgar/da...,"552,742,915"
AOS,AOS,'https://www.sec.gov/ix?doc=/Archives/edgar/da...,"25,905,876"
ABT,ABT,'https://www.sec.gov/ix?doc=/Archives/edgar/da...,"1,743,573,777"
ABBV,ABBV,'https://www.sec.gov/ix?doc=/Archives/edgar/da...,"1,768,480,508"
ACN,ACN,'https://www.sec.gov/ix?doc=/Archives/edgar/da...,"658,388,513"


In [128]:
df = df.assign(Raw_outstanding_share_vals=df["Oustanding_Shares"].str.replace(",", "").astype("int64"))
df.head()

,Symbols,Source_urls,Oustanding_Shares,Raw_outstanding_share_vals
MMM,MMM,'https://www.sec.gov/ix?doc=/Archives/edgar/da...,"552,742,915",552742915
AOS,AOS,'https://www.sec.gov/ix?doc=/Archives/edgar/da...,"25,905,876",25905876
ABT,ABT,'https://www.sec.gov/ix?doc=/Archives/edgar/da...,"1,743,573,777",1743573777
ABBV,ABBV,'https://www.sec.gov/ix?doc=/Archives/edgar/da...,"1,768,480,508",1768480508
ACN,ACN,'https://www.sec.gov/ix?doc=/Archives/edgar/da...,"658,388,513",658388513


In [129]:
summed_duplicates = df[df["Symbols"].isin(df[df.duplicated()]["Symbols"])].groupby("Symbols", as_index=False).agg({"Raw_outstanding_share_vals":"sum"}).set_index("Symbols")
summed_duplicates

,Raw_outstanding_share_vals
Symbols,
FOX,604949238
GOOG,1194600000
NWS,764702976


In [130]:
for vals in df["Symbols"]:
    if vals in summed_duplicates.index:
        df.loc[vals, "Raw_outstanding_share_vals"] = summed_duplicates.loc[vals].values[0]

In [136]:
df.drop_duplicates(inplace=True)

In [137]:
df.duplicated().any()

False

In [139]:
df.head()

,Symbols,Source_urls,Oustanding_Shares,Raw_outstanding_share_vals
MMM,MMM,'https://www.sec.gov/ix?doc=/Archives/edgar/da...,"552,742,915",552742915
AOS,AOS,'https://www.sec.gov/ix?doc=/Archives/edgar/da...,"25,905,876",25905876
ABT,ABT,'https://www.sec.gov/ix?doc=/Archives/edgar/da...,"1,743,573,777",1743573777
ABBV,ABBV,'https://www.sec.gov/ix?doc=/Archives/edgar/da...,"1,768,480,508",1768480508
ACN,ACN,'https://www.sec.gov/ix?doc=/Archives/edgar/da...,"658,388,513",658388513


In [140]:
df.to_csv("Outstanding_shares.csv")